In [3]:
import os, re, json, glob
import pandas as pd
import requests
from tqdm import tqdm

CONTROLLED_CSV = "controlled_agents_qwen.csv" 
DEBATE_CSV     = "debate_results_qwen.csv"
JSONL_GLOB     = "topic_*_qwen.jsonl"
OUT_CSV        = "Qwen_dialogue_LLaMA_reflection_score.csv"

OLLAMA_URL  = "http://localhost:11434/api/generate"
LLAMA_MODEL = "llama3:8b"
TIMEOUT     = 90

def llama_agree_1to5(a_text, b_text, ctx):
    prompt = f"""You are an impartial evaluator.
Rate how much the two people agree with each other on a 1–5 scale,
based ONLY on the following {ctx} texts.

1 = totally disagree, 5 = totally agree.

A: {a_text}

B: {b_text}

Output ONLY a single digit 1-5."""
    try:
        r = requests.post(
            OLLAMA_URL,
            json={"model": LLAMA_MODEL, "prompt": prompt, "stream": False},
            timeout=TIMEOUT
        )
        r.raise_for_status()
        txt = r.json().get("response", "")
        m = re.search(r"\b([1-5])\b", txt)
        return int(m.group(1)) if m else None
    except Exception as e:
        print("⚠️ LLaMA error:", e)
        return None


controlled = pd.read_csv(CONTROLLED_CSV, encoding="utf-8")
for c in ["region","occupation","gender"]:
    if c in controlled.columns:
        controlled[c] = controlled[c].astype(str).str.strip().str.lower()
if "topic_preference" in controlled.columns:
    controlled["topic_preference"] = controlled["topic_preference"].astype(int)

jsonl_files = sorted(glob.glob(JSONL_GLOB))
print(f"✅ Found {len(jsonl_files)} topic files")

debate_df = pd.read_csv(DEBATE_CSV, encoding="utf-8")
debate_df.columns = [c.strip() for c in debate_df.columns]
for k in ["A_id","B_id","topic_id"]:
    if k not in debate_df.columns:
        raise ValueError(f"❌ {DEBATE_CSV} 缺少欄位 {k}")
debate_df["order_idx"] = range(len(debate_df))
debate_df = debate_df.astype({"A_id": int, "B_id": int, "topic_id": int})


def pick_pref_response(topic_id, pref, region, occupation, gender, age):
    sub = controlled[controlled["topic_id"] == int(topic_id)].copy()
    if "topic_preference" in sub.columns and pd.notna(pref):
        sub = sub[sub["topic_preference"] == int(pref)]
    if "region" in sub.columns and region:
        sub = sub[sub["region"] == str(region).strip().lower()]
    if "occupation" in sub.columns and occupation and len(sub) > 1:
        sub = sub[sub["occupation"] == str(occupation).strip().lower()]
    if "gender" in sub.columns and gender and len(sub) > 1:
        sub = sub[sub["gender"] == str(gender).strip().lower()]
    if "age" in sub.columns and pd.notna(age) and len(sub) > 1:
        sub_exact = sub[sub["age"] == int(age)]
        if len(sub_exact) > 0:
            sub = sub_exact
    if len(sub) == 0:
        return None
    return str(sub.iloc[0]["Preference_Response"])


rows = []
for jf in tqdm(jsonl_files, desc="📂 Scoring Qwen dialogues (LLaMA judge + Reflection)"):
    with open(jf, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                d = json.loads(line)
            except Exception:
                continue

            topic_id = int(d.get("topic_id"))
            agents   = d.get("agents", [])
            rounds   = d.get("rounds", [])

            if not isinstance(agents, list) or len(agents) < 2:
                continue
            A, B = agents[0], agents[1]

            A_id = int(A.get("id"))
            B_id = int(B.get("id"))

            A_pref = A.get("pref"); B_pref = B.get("pref")
            A_region = A.get("region");  B_region = B.get("region")
            A_job    = A.get("occupation"); B_job = B.get("occupation")
            A_gender = A.get("gender"); B_gender = B.get("gender")
            A_age    = A.get("age");    B_age    = B.get("age")

            A_pref_resp = pick_pref_response(topic_id, A_pref, A_region, A_job, A_gender, A_age)
            B_pref_resp = pick_pref_response(topic_id, B_pref, B_region, B_job, B_gender, B_age)
            pre = llama_agree_1to5(A_pref_resp or "", B_pref_resp or "", "preference statements")

            A_reflection = d.get("reflection_A", "")
            B_reflection = d.get("reflection_B", "")

            if not A_reflection and rounds:
                A_reflection = rounds[-1].get("A", "")
            if not B_reflection and rounds:
                B_reflection = rounds[-1].get("B", "")

            post = llama_agree_1to5(A_reflection, B_reflection, "post-debate reflection")
            delta = (post - pre) if (pre is not None and post is not None) else None

            round_cols = {}
            for i, r in enumerate(rounds, 1):
                round_cols[f"round{i}_A"] = r.get("A", "")
                round_cols[f"round{i}_B"] = r.get("B", "")

            rows.append({
                "topic_id": topic_id,
                "A_id": A_id, "B_id": B_id,
                "A_pref": A_pref, "B_pref": B_pref,
                "A_region": A_region, "B_region": B_region,
                "A_occupation": A_job, "B_occupation": B_job,
                "A_pref_response": A_pref_resp,
                "B_pref_response": B_pref_resp,
                "A_reflection": A_reflection,
                "B_reflection": B_reflection,
                "pre_agreement": pre,
                "post_agreement": post,
                "Δagreement": delta,
                **round_cols
            })

df = pd.DataFrame(rows)

for c in ["topic_id","A_id","B_id"]:
    df[c] = df[c].astype(int)
ordered = pd.merge(
    debate_df[["topic_id","A_id","B_id","order_idx"]],
    df,
    on=["topic_id","A_id","B_id"],
    how="left",
    validate="one_to_one"
).sort_values("order_idx").drop(columns="order_idx").reset_index(drop=True)


ordered.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ Done! Saved {len(ordered)} rows to {OUT_CSV}")

missing = ordered[ordered.isna().any(axis=1)][["topic_id","A_id","B_id"]].head()
if len(missing):
    print(missing)

✅ Found 6 topic files


📂 Scoring Qwen dialogues (LLaMA judge + Reflection): 100%|█| 6/6 [06:42<00:00, 

✅ Done! Saved 90 rows to Qwen_dialogue_LLaMA_reflection_score.csv
